In [19]:
import os
%pwd

'/Users/hh/MLops/dataScienceProject'

In [20]:
os.chdir("/Users/hh/MLops/dataScienceProject")

In [21]:
%pwd

'/Users/hh/MLops/dataScienceProject'

In [22]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path 

In [23]:
from src.datascience.constants import *

from src.datascience.utils.common import *

In [24]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH,
                 schema_file_path=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)
        self.schema=read_yaml(schema_file_path)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion(self) -> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config



In [25]:
import requests
from urllib import request
from src.datascience import logger
import zipfile

In [26]:
## component - Data Ingestion
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download, with following info {headers}")
        else:
            logger.info("File already exists!")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)



        

In [27]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2025-08-05 15:18:43,310: INFO: common: yaml file config/config.yaml is loaded successfully]
[2025-08-05 15:18:43,311: INFO: common: yaml file params.yaml is loaded successfully]
[2025-08-05 15:18:43,312: INFO: common: yaml file schema.yaml is loaded successfully]
[2025-08-05 15:18:43,313: INFO: common: created directory at artifacts]
[2025-08-05 15:18:43,314: INFO: common: created directory at artifacts/data_ingestion]
[2025-08-05 15:18:44,480: INFO: 3207460213: artifacts/data_ingestion/data.zip download, with following info Connection: close
Content-Length: 36045
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "267bbee00662c6f35a0ab600ffa451ce5c1b11c6268ccb5e883438dcae51a3a4"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 24E6:1C1120:835DC2:9769F0:6891F6A5
Accept-Ranges: bytes
Date: Tue, 